In [1]:
# logger, i'll use one logger for this script

import logging
import sys

logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')

stdout_handler = logging.StreamHandler(sys.stdout)
stdout_handler.setLevel(logging.DEBUG)
stdout_handler.setFormatter(formatter)

file_handler = logging.FileHandler('logs.log')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)


logger.addHandler(file_handler)
logger.addHandler(stdout_handler)

In [2]:
# the goal is to scrape all movie ids. but all movies page has 660k entries
# clicking a button for 13000 times is not reliable so we need to scrape movies
# in batches. we can use various filtering options.
# filtering options:
# - year
# - imdb rating

imdb_link = "https://www.imdb.com/search/title/?title_type=feature"

increment = 0.1
imdb_links = []
for i in range(1, 10):
    for j in range(5):
        jj = j / 5
        imdb_links.append(
            "https://www.imdb.com/search/title/?title_type=feature&user_rating=" +
            str(round(i + jj, 1)) + "," + str(round(i + jj + increment, 1))
        )
# there actually are movies with 10 ratings
imdb_links.append("https://www.imdb.com/search/title/?title_type=feature&user_rating=10,")

print(imdb_links)
for link in imdb_links:
    print(link)

# i shouldnt be doing imdb ratings because ratings can change while the scraping is happening
# i need to do this with dates. which would be a lot better because that way updating the
# database is just a breeze



['https://www.imdb.com/search/title/?title_type=feature&user_rating=1.0,1.1', 'https://www.imdb.com/search/title/?title_type=feature&user_rating=1.2,1.3', 'https://www.imdb.com/search/title/?title_type=feature&user_rating=1.4,1.5', 'https://www.imdb.com/search/title/?title_type=feature&user_rating=1.6,1.7', 'https://www.imdb.com/search/title/?title_type=feature&user_rating=1.8,1.9', 'https://www.imdb.com/search/title/?title_type=feature&user_rating=2.0,2.1', 'https://www.imdb.com/search/title/?title_type=feature&user_rating=2.2,2.3', 'https://www.imdb.com/search/title/?title_type=feature&user_rating=2.4,2.5', 'https://www.imdb.com/search/title/?title_type=feature&user_rating=2.6,2.7', 'https://www.imdb.com/search/title/?title_type=feature&user_rating=2.8,2.9', 'https://www.imdb.com/search/title/?title_type=feature&user_rating=3.0,3.1', 'https://www.imdb.com/search/title/?title_type=feature&user_rating=3.2,3.3', 'https://www.imdb.com/search/title/?title_type=feature&user_rating=3.4,3.5'

In [3]:
from datetime import date


current_date = date.today()
y, m, d = str(current_date).split("-")
y = int(y)
m = int(m)
d = int(d)

# do the thing in one month periods
# 02-01 to 01-02
# 02-02 to 01-03
# 02-03 to 01-04
# ...
# 02-11 to 01-12
# 02-12 to 01-01-nextyear

imdb_links_2 = []

# spaghetti code

def generate_imdb_links_by_release_dates(date_range):
    links = []
    for year in range(date_range[0], date_range[1]):
        for i in range(12):
            from_day = 2
            from_month = i + 1
            from_year = year

            to_day = 1
            to_month = i + 2
            to_year = year

            if to_month == m and to_year == y:
                to_day = d

            if to_month == 13:
                to_month = 1
                to_year += 1

            #print(f"{from_day:02d}-{from_month:02d}-{from_year} to {to_day:02d}-{to_month:02d}-{to_year}")
            links.append(f"https://www.imdb.com/search/title/?title_type=feature&release_date={from_year}-{from_month:02d}-{from_day:02d},{to_year}-{to_month:02d}-{to_day:02d}")

            if to_month == m and to_year == y:
                break
    return links

imdb_links_2 = generate_imdb_links_by_release_dates([2020, 2025])

for link in imdb_links_2:
    print(link)


https://www.imdb.com/search/title/?title_type=feature&release_date=2000-01-02,2000-02-01
https://www.imdb.com/search/title/?title_type=feature&release_date=2000-02-02,2000-03-01
https://www.imdb.com/search/title/?title_type=feature&release_date=2000-03-02,2000-04-01
https://www.imdb.com/search/title/?title_type=feature&release_date=2000-04-02,2000-05-01
https://www.imdb.com/search/title/?title_type=feature&release_date=2000-05-02,2000-06-01
https://www.imdb.com/search/title/?title_type=feature&release_date=2000-06-02,2000-07-01
https://www.imdb.com/search/title/?title_type=feature&release_date=2000-07-02,2000-08-01
https://www.imdb.com/search/title/?title_type=feature&release_date=2000-08-02,2000-09-01
https://www.imdb.com/search/title/?title_type=feature&release_date=2000-09-02,2000-10-01
https://www.imdb.com/search/title/?title_type=feature&release_date=2000-10-02,2000-11-01
https://www.imdb.com/search/title/?title_type=feature&release_date=2000-11-02,2000-12-01
https://www.imdb.com/

In [4]:
# init browser

from math import ceil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

import time 
  
from selenium.webdriver.common.keys import Keys 

browser = webdriver.Chrome()
browser.maximize_window()

In [5]:
def scrape_ids_from_link(browser, link):
    browser.get(link)

    browser.implicitly_wait(10)

    # get the total number of movies from the page
    try:
        element = browser.find_element(By.CLASS_NAME, "ipc-page-grid").find_elements(By.CLASS_NAME, "ipc-page-grid__item")[1]
    except Exception as e:
        logger.info("there are no movies. maybe??")
        #logger.exception(e)
        #logger.info("EXITING")
        #exit(1)
        return -1


    number_of_movies_str = element.text.split("\n")[0].split(" ")[2].replace(",", "")
    print(number_of_movies_str)
    number_of_movies = int(number_of_movies_str)
    print(number_of_movies)

    # on one click, the page loads 50 more movies
    number_of_max_clicks = ceil(number_of_movies / 50) - 1

    print("there are " + number_of_movies_str + " movies. we need to click " + str(number_of_max_clicks) + " in total. haha")


    # click until we load all the movies
    # TODO: implement falllbacks on possible errors.
    # TODO: implement sanity check while clicking buttons.

    # b is the "see more" button at the bottom of the page
    b = browser.find_elements(By.CLASS_NAME, "ipc-see-more")
    print(b)
    for i in range(number_of_max_clicks):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(2)
        while True:
            try:
                b[0].click()
                break
            except:
                logger.error("something bad happened while clicking. trying again")
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(2)

        time.sleep(2)
        #print("[LOG] " + str(i) + " out of " + str(number_of_max_clicks) + " completed. %" + str(i / number_of_max_clicks))
        logger.info(str(i + 1) + " out of " + str(number_of_max_clicks) + " completed. %" + str((i + 1) / number_of_max_clicks * 100))

    logger.info("done. maybe??")
    return 0

In [6]:
i = 0
final_links = imdb_links_2

for link in final_links:
    i += 1
    link_date = link.split("release_date=")[1]
    links_file = open(f"./imdb_ids/{link_date}.txt", "w")
    
    logger.info(f"START[{i}/{len(final_links)}]! attemting to scrape links for date [{link_date}]")

    ret = 0
    while True:
        try:
            ret = scrape_ids_from_link(browser, link)
            if ret == 0:
                logger.info("clicking completed.")
            break
        except:
            logger.error(f"failed to scrape links for date [{link_date}]. trying again")
            pass
        

    # get all ids from the webpage
    # TODO: remember implementing sanity checks

    # im just assuming this works here
    if ret == 0:
        logger.info("getting all the links and saving them to a file")
        for elem in browser.find_elements(By.CLASS_NAME, "ipc-title-link-wrapper"):
            links_file.write(elem.get_attribute("href"))
            links_file.write("\n")
        logger.info("saved links to file.")
    else:
        logger.info("no links found.")


2024-02-04 07:49:45,010 | INFO | START[1/80]! attemting to scrape links for date [2017-06-02,2017-07-01]
1324
1324
there are 1324 movies. we need to click 26 in total. haha
[<selenium.webdriver.remote.webelement.WebElement (session="1c5306b6db4fc57f65e125479e3ea953", element="FFA4C5E0769BA91E1B1651A9D3E7F02E_element_68")>]
2024-02-04 07:49:53,443 | INFO | 1 out of 26 completed. %0.038461538461538464
2024-02-04 07:49:57,521 | INFO | 2 out of 26 completed. %1.0384615384615385
2024-02-04 07:50:01,620 | INFO | 3 out of 26 completed. %2.0384615384615383
2024-02-04 07:50:05,727 | INFO | 4 out of 26 completed. %3.0384615384615383
2024-02-04 07:50:09,814 | INFO | 5 out of 26 completed. %4.038461538461538
2024-02-04 07:50:13,922 | INFO | 6 out of 26 completed. %5.038461538461538
2024-02-04 07:50:18,035 | INFO | 7 out of 26 completed. %6.038461538461538
2024-02-04 07:50:22,138 | INFO | 8 out of 26 completed. %7.038461538461538
2024-02-04 07:50:26,244 | INFO | 9 out of 26 completed. %8.0384615384

In [7]:
# we got the links from year 2000 to 2024, enough for now
# how do we scrape those links one by one

# we can just use mechanicalsoup for the actual data scraping,
# since we dont need to interact with any buttons or such


import mechanicalsoup

mbrowser = mechanicalsoup.StatefulBrowser(
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
}

#url = "https://www.imdb.com/title/tt0099685"
#urls = [
#    "https://www.imdb.com/title/tt0099685/",
#    "https://www.imdb.com/title/tt1130884",
#    "https://www.imdb.com/title/tt1375666",
#    "https://www.imdb.com/title/tt0468569",
#]


In [8]:
# get the links from a file
read_links_file = open("./imdb_ids/2000-01-02,2000-02-01.txt", "r")

links = [link for link in read_links_file.read().split("\n") if link != ""]

In [10]:
for url in links:
    test_file = open("test_file.txt", "w")
    page = mbrowser.open(
        url,
    )
    
    print(page.soup.find("span", {"class": "hero__primary-text"}).text)
    #for e in page.soup.findAll("a", {"class": "ipc-link"}):
    #    print(e.text)
        
    
    print("year: " + page.soup.findAll("a", {"class": "ipc-link"})[6].text)
    print("r rating: " + page.soup.findAll("a", {"class": "ipc-link"})[7].text)
    print("imdb rating " + page.soup.findAll("div", attrs={"data-testid": "hero-rating-bar__aggregate-rating__score"})[0].find("span").text)

    # there are movies that has no metascore.
    try:
        print("metascore: " + page.soup.findAll("span", {"class": "metacritic-score-box"})[0].text)
    except:
        print("metascore not found.")
    
    #for e in page.soup.findAll("li", {"class": "ipc-inline-list__item"}):
    #    print(e.text)
    
    print("time " + page.soup.findAll("li", {"class": "ipc-inline-list__item"})[6].text)
    #for e in page.soup.findAll("a", {"class": "ipc-metadata-list-item__list-content-item"}):
    #    print(e.text)
    
    #print("director: " + page.soup.findAll("a", {"class": "ipc-metadata-list-item__list-content-item"})[0].text)
    #print("writers: " + page.soup.findAll("a", {"class": "ipc-metadata-list-item__list-content-item"})[0].text)
    #print(page.soup.findAll("li", {"class": "ipc-metadata-list__item"})[0].find("ul", {"class": "ipc-inline-list"}).findAll("a"))
    
    print("director(s): ", end="")
    for elem in page.soup.findAll("li", {"class": "ipc-metadata-list__item"})[0].find("ul", {"class": "ipc-inline-list"}).findAll("a"):
        print(elem.text, end=", ")
    print()
    
    print("writer(s): ", end="")
    for elem in page.soup.findAll("li", {"class": "ipc-metadata-list__item"})[1].find("ul", {"class": "ipc-inline-list"}).findAll("a"):
        print(elem.text, end=", ")
    print()
    
    print("star(s): ", end="")
    for elem in page.soup.findAll("li", {"class": "ipc-metadata-list__item"})[2].find("ul", {"class": "ipc-inline-list"}).findAll("a"):
        print(elem.text, end=", ")
    print()
    print()
    print()
    print()
        

American Psycho
year: 2000
r rating: 18+
imdb rating 7.6
metascore: 64
time 1h 42m
director(s): Mary Harron, 
writer(s): Bret Easton Ellis, Mary Harron, Guinevere Turner, 
star(s): Christian Bale, Justin Theroux, Josh Lucas, 



You Can Count on Me
year: 2000
r rating: R
imdb rating 7.5
metascore: 85
time 1h 51m
director(s): Kenneth Lonergan, 
writer(s): Kenneth Lonergan, 
star(s): Laura Linney, Matthew Broderick, Amy Ryan, 



Tra(sgre)dire
year: 2000
r rating: Not Rated
imdb rating 5.4
metascore not found.
time 1h 31m
director(s): Tinto Brass, 
writer(s): Tinto Brass, Carla Cipriani, Nicolaj Pennestri, 
star(s): Yuliya Mayarchuk, Jarno Berardi, Francesca Nunzi, 



Next Friday
year: 2000
r rating: R
imdb rating 6.1
metascore: 41
time 1h 38m
director(s): Steve Carr, 
writer(s): Ice Cube, DJ Pooh, 
star(s): Ice Cube, Mike Epps, Justin Pierce, 



Love & Basketball
year: 2000
r rating: PG-13
imdb rating 7.2
metascore: 72
time 2h 4m
director(s): Gina Prince-Bythewood, 
writer(s): Gina Pr

KeyboardInterrupt: 